In [2]:
from streaming import LocalDataset
from streaming import MDSWriter
from silero_vad import load_silero_vad
from datasets import Audio
from glob import glob
from tqdm import tqdm
import numpy as np
import IPython.display as ipd
import soundfile as sf
import json
import pickle
import os
import re
import torch
import mp


sr = 16000
window_size_samples = 512

def segment_texts(text):
    pattern = r'<\|(\d+(?:\.\d+)?)\|>(.*?)(?=<\||\Z)'
    
    matches = re.findall(pattern, text, re.DOTALL)
    
    result = []
    for i, (start_time, content) in enumerate(matches):
        content = content
        if content:
            end_time = matches[i+1][0] if i+1 < len(matches) else start_time
            result.append((float(start_time), float(end_time), f"<|{start_time}|>{content}<|{end_time}|>"))
    
    return result



class Pointer:
    def __init__(self, filename):
        self.filename = filename
        self.index = -1

    def _save(self):
        with open(self.filename, 'wb') as fopen:
            pickle.dump(self.index, fopen)

    def increment(self):
        self.index += 1
        self._save()

    def load(self):
        if not os.path.exists(self.filename):
            return
        with open(self.filename, 'rb') as fopen:
            self.index = pickle.load(fopen)

def loop(ranged):
    
    audio = Audio(sampling_rate=sr)
    model = load_silero_vad(onnx=True)
    ranged, index = ranged
    filename = f'vad-audio-indon-{index}.jsonl'
    fopen_l = open(filename, 'a')
    pointer = Pointer(f'{filename}.pickle')
    pointer.load()
    dataset = LocalDataset('mosaic-indon')
    n = 0
    for i in tqdm(ranged):
        if n >= pointer.index:
            entry = dataset[i]
                    
            audio_filename = entry['audio_filename']
            if not os.path.exists(audio_filename):
                continue

            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            label = entry['new_text']
            label_en = entry['new_text_en']
            label_ms = entry['new_text_ms']
            segments = segment_texts(label)

            r = 0

            for k in range(len(segments)):

                segment = segments[k]
                segment_text = re.sub(r'<\|.*?\|>', '', segment[2])

                if k + 1 < len(segments):
                    segment_text_2 = re.sub(r'<\|.*?\|>', '', segments[k+1][2])
                    if segment_text == segment_text_2:
                        label = label.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")
                        if len(label_ms):
                            label_ms = label_ms.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")

                        if len(label_en):
                            label_en = label_en.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")

                        r += 1
                        continue


                segment_audio = torch.Tensor(y[int(segment[0] * sr): int(segment[1] * sr)])
                audio_length_samples = len(segment_audio)

                speech_probs = []

                for current_start_sample in range(0, audio_length_samples, window_size_samples):
                    chunk = segment_audio[current_start_sample: current_start_sample + window_size_samples]
                    if len(chunk) < window_size_samples:
                        chunk = torch.nn.functional.pad(chunk, (0, int(window_size_samples - len(chunk))))
                    speech_prob = model(chunk, sr).item()
                    speech_probs.append(speech_prob)

                vad_prob = np.mean(speech_probs)

                if vad_prob < 0.001:
                    label = label.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")
                    if len(label_ms):
                        label_ms = label_ms.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")
                    if len(label_en):
                        label_en = label_en.replace(segment[2], f"<|{segment[0]}|><|{segment[1]}|>")

            
            entry['new_text'] = label  
            entry['new_text_en'] = label_en
            entry['new_text_ms'] = label_ms  
            entry['index'] = n
            fopen_l.write(f'{json.dumps(entry)}\n')
            fopen_l.flush()
            
            pointer.index = n
            pointer._save()
        
        n += 1

In [3]:
dataset = LocalDataset('mosaic-indon')
len(dataset)

320934

In [5]:
mp.multiprocessing(range(len(dataset)), loop, cores = 15, returned = False)

In [8]:
files = glob('vad-audio-indon-*.jsonl')
files

['vad-audio-indon-15.jsonl',
 'vad-audio-indon-1.jsonl',
 'vad-audio-indon-7.jsonl',
 'vad-audio-indon-8.jsonl',
 'vad-audio-indon-6.jsonl',
 'vad-audio-indon-5.jsonl',
 'vad-audio-indon-13.jsonl',
 'vad-audio-indon-12.jsonl',
 'vad-audio-indon-4.jsonl',
 'vad-audio-indon-3.jsonl',
 'vad-audio-indon-2.jsonl',
 'vad-audio-indon-10.jsonl',
 'vad-audio-indon-9.jsonl',
 'vad-audio-indon-11.jsonl',
 'vad-audio-indon-0.jsonl',
 'vad-audio-indon-14.jsonl']

In [12]:
with open('indonesian-stt.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                
                if len(l['new_text_en']):
                    d = {
                        'audio_filename': l['audio_filename'],
                        'text': l['new_text_en']
                    }
                    fopen_l.write(f'{json.dumps(d)}\n')
                    fopen_l.flush()
                    
                if len(l['new_text_ms']):
                    d = {
                        'audio_filename': l['audio_filename'],
                        'text': l['new_text_ms']
                    }
                    fopen_l.write(f'{json.dumps(d)}\n')
                    fopen_l.flush()
                    
                d = {
                    'audio_filename': l['audio_filename'],
                    'text': l['new_text']
                }
                fopen_l.write(f'{json.dumps(d)}\n')
                fopen_l.flush()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.44it/s]


In [13]:
!wc -l indonesian-stt.jsonl

943282 indonesian-stt.jsonl


In [14]:
!ls -lh indonesian-stt.jsonl

-rw-rw-r-- 1 husein husein 547M Okt  24 11:31 indonesian-stt.jsonl


In [15]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='indonesian-stt.jsonl',
    path_in_repo="indonesian-stt.jsonl",
    repo_id="mesolitica/malaysian-stt",
    repo_type="dataset",
)

indonesian-stt.jsonl:   0%|          | 0.00/573M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/malaysian-stt/commit/61dad824d651bcb58937f66a4fa65d328aa5e9ea', commit_message='Upload indonesian-stt.jsonl with huggingface_hub', commit_description='', oid='61dad824d651bcb58937f66a4fa65d328aa5e9ea', pr_url=None, pr_revision=None, pr_num=None)